In [1]:
import torch

# Check if MPS (Metal Performance Shaders) is available and set the device accordingly
device = torch.device("mps" if torch.backends.mps.is_built() else "cpu")
print("Using device:", device)

Using device: mps
